# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
df_imm = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
feats_imm = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
df_ld_imm = df_imm['Subject ID'].value_counts().to_frame()
df_imm['Is longitudinal?'] = False
df_imm.loc[df_imm['Subject ID'].isin(df_ld_imm.index[df_ld_imm['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_imm = df_imm.loc[(df_imm['Status'] == 'Control'), :]
df_imm.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_imm['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
df_imm_ppr = pd.read_excel(f"{path}/data/immuno/models/SImAge/data.xlsx", index_col="sample_id")
ids_imm_ppr = df_imm_ppr.index[df_imm_ppr['Status'] == 'Control'].values
feats_imm_ppr = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values

epi_suffix = "_harm"
df_epi = pd.read_excel(f"{path}/pheno.xlsx", index_col="index")
df_epi.index.name = "index"
df_epi.drop(["I64_old", "I1_duplicate"], inplace=True)
df_epi.rename(columns={'Subject_ID': 'Subject ID'}, inplace=True)
df_ld_epi = df_epi['Subject ID'].value_counts().to_frame()
df_epi['Is longitudinal?'] = False
df_epi.loc[df_epi['Subject ID'].isin(df_ld_epi.index[df_ld_epi['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_epi = df_epi.loc[(df_epi['Status'] == 'Control'), :]
df_epi.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_epi['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
ids_epi_ppr = pd.read_excel(f"{path}/data/GSE234461/samples.xlsx", index_col=0).index.values

path_save = f"{path}/special/059_imm_data_selection"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

# Select samples

## Filter Yakutian samples with small SImAge MAE value

In [ ]:
thld_mae = 8.05

df_imm_yak = df_imm.loc[df_imm['Region'] == 'Yakutia']
df_imm_yak.sort_values(["|SImAge acceleration|"], ascending=[True], inplace=True)
df_imm_yak['|SImAge acceleration| cumsum'] = df_imm_yak['|SImAge acceleration|'].expanding().mean()
ids_imm_yak = df_imm_yak.index[df_imm_yak['|SImAge acceleration| cumsum'] < thld_mae].values

df_imm_cnt = df_imm.loc[(df_imm['Region'] == 'Central') & ~df_imm.index.isin(ids_imm_ppr), :]
df_imm_cnt.sort_values(["|SImAge acceleration|"], ascending=[True], inplace=True)
df_imm_cnt['|SImAge acceleration| cumsum'] = df_imm_cnt['|SImAge acceleration|'].expanding().mean()
ids_imm_cnt = df_imm_cnt.index[df_imm_cnt['|SImAge acceleration| cumsum'] < thld_mae].values

mae_imm_ppr = mean_absolute_error(df_imm.loc[ids_imm_ppr, 'Age'].values, df_imm.loc[ids_imm_ppr, 'SImAge'].values)
mae_imm_yak = mean_absolute_error(df_imm.loc[ids_imm_yak, 'Age'].values, df_imm.loc[ids_imm_yak, 'SImAge'].values)
mae_imm_cnt = mean_absolute_error(df_imm.loc[ids_imm_cnt, 'Age'].values, df_imm.loc[ids_imm_cnt, 'SImAge'].values)

## Define samples for immunology and epigenetics

In [ ]:
ids_imm = list(set.union(set(ids_imm_ppr), set(ids_imm_yak)))
# ids_imm = list(set.union(set(ids_imm_ppr), set(ids_imm_yak), set(ids_imm_cnt)))
ids_epi_ppr = ids_epi_ppr
ids_epi_full = df_epi.index.values

epi_types = {
    'paper_only': ids_epi_ppr,
    'full': ids_epi_full
}

## Compare immunology and epigenetics samples

In [ ]:
for epi_type, ids_epi in epi_types.items():

    pathlib.Path(f"{path_save}/imm_vs_epi/{epi_type}").mkdir(parents=True, exist_ok=True)
    
    sections = get_sections([set(ids_imm), set(ids_epi)])
    for sec in sections:
        df_sec = pd.DataFrame(index=list(sections[sec]))
        df_sec.to_excel(f"{path_save}/imm_vs_epi/{epi_type}/{sec}.xlsx", index_label='index')
    
    fig, ax = plt.subplots()
    venn = venn2(
        subsets=(set(ids_imm), set(ids_epi)),
        set_labels = ('Imm', 'Epi'),
        set_colors=('r', 'g'),
        alpha = 0.5
    )
    venn2_circles(subsets=(set(ids_imm), set(ids_epi)))
    for text in venn.set_labels:
        text.set_fontsize(16)
    for text in venn.subset_labels:
        text.set_fontsize(25)
    plt.savefig(f"{path_save}/imm_vs_epi/{epi_type}/venn.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_save}/imm_vs_epi/{epi_type}/venn.pdf", bbox_inches='tight')
    plt.clf()